# из идей
- для генеративного метода хорошо бы лосс, который не будет учитывать порядок
- использовать для оценки все препараты или только основного контекста, или только популярные
- использовать не только симптомы, но и заболевания
- рейтинг слишком дисбалансный получается, может, если только для популярных считать, будет лучше

In [1]:
import json
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
from sklearn.model_selection import KFold

In [2]:
USE_ONLY_MAIN_CONTEXT = False  # используем все сущности, или только основные
CORPUS_PATH = "/s/ls4/users/grartem/SAG_MED/RelationExtraction/data/RDRS3_11_07_2022_clean/raw/medNorm_11072022_mostPopNorm.jsonlines"
corpus_ent_only = []
with open(CORPUS_PATH, "r") as f:
    for line in f:
        docData = json.loads(line)
        corpus_ent_only.append(docData["objects"]["MedEntity"])

In [3]:
if USE_ONLY_MAIN_CONTEXT:
    # удаляем все сущности, которые не принадлежат основному контексту
    for d_i, doc in enumerate(corpus_ent_only):
        new_entities_list = []
        for entity in doc:
            contexts = entity["Context"].split(",")
            if '1' in contexts:
                new_entities_list.append(entity)
        corpus_ent_only[d_i] = new_entities_list
            

In [4]:
# подгружаем таблицу. где есть стандартизированные варианты препаратов
Drug_normalization = pd.read_excel("../data/external/Drugname_2022.xls")
Drug_normalization = Drug_normalization.set_index("mention")
# превращаем её в словарь, чтоб можно было легко доставать стандартизированные формы для упоминаний
Drug_normalization_dict = Drug_normalization.transpose().to_dict()

# таблица препарат-адры-качество
Колонки:
- Drug - название препарата из отзыва, замененное на стандартизированное название из таблицы external/Drugname_2022.xls
- ADR_all - список всех нежелательных реакций (приведенных к MedDRA) которые встречались в одном отзыве с препаратом, упорядочены по убыванию популярности появления в отзывах с этим препаратом.
- ADR_top5 - топ 5 по популярности ADR
- Quality - оценка качества / рейтинг препарата (см. ниже)
- Fold - номер фолда (для кросс валидации)

Оценка качества / рейтинг: 
- хороший - отзывов с BNE-Pos больше, чем отзывов с ADE-Neg,Negated,ADR или Worse, ADR==0
- хороший, но есть адр - тоже самое, но ADR > 0
- плохой - отзывов с BNE-Pos не больше, чем отзывов с ADE-Neg,Negated,ADR или Worse, ADR==0
- плохой, частые побочные реакции  - плохой, и при этом ADR Больше чем NEG или Nagated
- плохой, не помогает  - плохой, и при этом Nagated Больше чем NEG или ADR


In [5]:
# Для каждого препарата формируем словарь-счетчик, считаем количество MedDRA-шных адров и разных типов эффектов
drug_adr_effects_dict = defaultdict(Counter)
for docData in corpus_ent_only:
    drugs = set()
    effects = set()
    adrs = set()
    # сначала смотрим, какие в отзыве упомянуты препараты
    for entity in docData:
        if entity.get("MedType", None)=="Drugname":
            drugs.add(Drug_normalization_dict[entity["text"].lower()]["standart"])
    # потом собираем упоминания эффектов и уникальные адры
    for entity in docData:
        if entity.get("DisType", None)=='NegatedADE':
            effects.add("NegatedADE")
        if entity.get("DisType", None)=='Worse':
            effects.add("Worse")
        if entity.get("DisType", None)=='ADE-Neg':
            effects.add("ADE-Neg")
        if entity.get("DisType", None)=='BNE-Pos':
            effects.add("BNE-Pos")
        if entity.get("MedEntityType", None)=='ADR':
            effects.add("ADR")
            if entity.get("MedDRA", "").strip() != "":
                adrs.add(entity["MedDRA"])
            else:
                adrs.add(entity["text"])
    # для каждого препарата добавляем информацию о появлении разных эффектов в отзыве с ним
    for drug in drugs:
        for adr in adrs:
            drug_adr_effects_dict[drug][adr]+=1
        if len(effects & set(["Worse", "ADE-Neg"])) > 0:
            drug_adr_effects_dict[drug]["Neg"] += 1
        if "BNE-Pos" in effects:
            drug_adr_effects_dict[drug]["Pos"] += 1
        if "ADR" in effects:
            drug_adr_effects_dict[drug]["ADR"] += 1
        if "NegatedADE" in effects:
            drug_adr_effects_dict[drug]["Negated"] += 1

In [6]:
# теперь формируем таблицу
drug_adr_quality_table = []
for k,v in drug_adr_effects_dict.items():
    # определяем качество по количеству отзывов с разными категориями эффектов
    quality = None
    if v["Pos"] > v["Neg"]+v["ADR"]+v["Negated"]:
        quality = "хороший"
        if v["ADR"]>0:
            quality = "хороший, но есть побочные реакции"
    else:
        if v["ADR"]>v["Neg"]+v["Negated"]:
            quality = "плохой, частые побочные реакции"
        elif v["Negated"]> v["ADR"]+v["Neg"]:
            quality = "плохой, не помогает"
        else:
            quality = "плохой"
    assert quality is not None
    # сортируем адры по количеству отзывов с ними
    sorted_adr_list = sorted([(x,v) for x,v in v.items() if x not in ["Neg", "Pos", "ADR", "Negated"]], key=lambda x: x[1], reverse=True)
    drug_adr_quality_table.append({
        "Drug": k,
        "ADR_all": ", ".join([x[0] for x in sorted_adr_list]),
        "ADR_top5": ", ".join([x[0] for x in sorted_adr_list][:5]),
        "Quality": quality
    })
drug_adr_quality_table = pd.DataFrame(drug_adr_quality_table)

In [7]:
# бьем на фолды
drug_adr_quality_table['Fold'] = 1
kf = KFold(n_splits=5, shuffle=True)
for i, (train_indexes, test_indexes) in enumerate(kf.split(drug_adr_quality_table.values)):
    drug_adr_quality_table.loc[test_indexes, "Fold"] = i + 1

In [8]:
# проверяем, как распределены классы качества по фолдам
for i in range(1, 6):
    print(drug_adr_quality_table[drug_adr_quality_table["Fold"]==i]["Quality"].value_counts())

плохой, частые побочные реакции      103
плохой                                79
хороший                               43
плохой, не помогает                   23
хороший, но есть побочные реакции     15
Name: Quality, dtype: int64
плохой, частые побочные реакции      117
плохой                                73
хороший                               31
плохой, не помогает                   22
хороший, но есть побочные реакции     20
Name: Quality, dtype: int64
плохой, частые побочные реакции      102
плохой                                77
хороший                               36
плохой, не помогает                   29
хороший, но есть побочные реакции     19
Name: Quality, dtype: int64
плохой, частые побочные реакции      120
плохой                                74
хороший                               28
плохой, не помогает                   21
хороший, но есть побочные реакции     20
Name: Quality, dtype: int64
плохой, частые побочные реакции      105
плохой                     

In [9]:
if USE_ONLY_MAIN_CONTEXT:
    drug_adr_quality_table.to_csv("../data/interim/drugs_quality_adr_mainContext.csv", index=False)
else:
    drug_adr_quality_table.to_csv("../data/interim/drugs_quality_adr.csv", index=False)
drug_adr_quality_table

,Drug,ADR_all,ADR_top5,Quality,Fold
0,оксолин,"Дискомфорт в носу, Заложенность носа, Гиперсек...","Дискомфорт в носу, Заложенность носа, Гиперсек...","хороший, но есть побочные реакции",2
1,акдс,"Пирексия, Повышенная температура тела, Инфекци...","Пирексия, Повышенная температура тела, Инфекци...","плохой, частые побочные реакции",2
2,цефекон д,"Дерматит, Смерть, Кашель, Астма, Нарушение со ...","Дерматит, Смерть, Кашель, Астма, Нарушение со ...",плохой,1
3,зиртек,"Кашель, Сомнолентность, Дерматит, Смерть, Астм...","Кашель, Сомнолентность, Дерматит, Смерть, Астма",плохой,2
4,пентаксим,"Пирексия, Повышенная температура тела, Боль в ...","Пирексия, Повышенная температура тела, Боль в ...","плохой, частые побочные реакции",4
...,...,...,...,...,...
1309,леденцы лакричные,"давить сердце, Повышение артериального давления","давить сердце, Повышение артериального давления","плохой, частые побочные реакции",2
1310,диафлекс ромфарм,"Абдоминальный дискомфорт, Диарея","Абдоминальный дискомфорт, Диарея","плохой, частые побочные реакции",5
1311,диафлекс,"Абдоминальный дискомфорт, Диарея","Абдоминальный дискомфорт, Диарея","плохой, частые побочные реакции",3
1312,целебрекс,"Заложенность носа, сжимание в сердце, Хроничес...","Заложенность носа, сжимание в сердце, Хроничес...",плохой,4


# таблица симптом-препараты
Колонки:
- indication симптом (приведенные к MedDRA) встречаемый в корпусе 
- drugs - список препаратов (стандартизированных по Drugname_2022.xls) встречаемые в отзывах с указанным симптомом, отсортированы по популярности
- drugs_top5 - оставлены только топ 5 препаратов
- drugs_best - оставлены только те препараты, которые имеют рейтинг "хороший"
- fold - номер фолда для кроссвалидации

In [10]:
# для каждого симптома (по меддре) делаем счетчик с препаратами и количеством отзывов в которых этот препарат появился с этим симптомом
indication_drug_dict = defaultdict(Counter)
for docData in corpus_ent_only:
    drugs = set()
    for entity in docData:
        if entity.get("MedType", None)=="Drugname":
            drugs.add(Drug_normalization_dict[entity["text"].lower()]["standart"])
    for entity in docData:
        if entity.get("DisType", None)=='Indication':
            if entity["MedDRA"].strip()!="":
                for drug in drugs:
                    indication_drug_dict[entity["MedDRA"]][drug] += 1
            else:
                for drug in drugs:
                    indication_drug_dict[entity["text"]][drug] += 1

In [11]:
indication_drug_table = []
for indication, drugs_counter in indication_drug_dict.items():
    # сортируем препараты по количеству отзывов в которых был упомянут с этим симптомом
    sorted_drugs = sorted([(x,v) for x,v in drugs_counter.items()], key=lambda x:x[1], reverse=True)
    # отбираем только хорошие препараты
    best_drugs = []
    for drug in sorted_drugs:
        quality = drug_adr_quality_table.loc[drug_adr_quality_table["Drug"]==drug[0], "Quality"]
        if len(quality)==0:
            continue
        quality = quality.values[0]
        #if quality in ["хороший", "хороший, но есть побочные реакции"]:
        if quality in ["хороший"]:
            best_drugs.append(drug[0])
    indication_drug_table.append({
        "indication": indication,
        "drugs": ", ".join([x[0] for x in sorted_drugs]),
        "drugs_top5": ", ".join([x[0] for x in sorted_drugs[:5]]),
        "drugs_best": ", ".join(best_drugs) if len(best_drugs)>0 else "нет хороших лекарств"
    })
indication_drug_table = pd.DataFrame(indication_drug_table)

In [12]:
# бьем на фолды
indication_drug_table['fold'] = 1
kf = KFold(n_splits=5, shuffle=True)
for i, (train_indexes, test_indexes) in enumerate(kf.split(indication_drug_table.values)):
    indication_drug_table.loc[test_indexes, "fold"] = i + 1

In [13]:
# Проверяем, сколько в каждом фолде меток "нет хороших"
for i in range(1, 6):
    print("всего в фолде", len(indication_drug_table[indication_drug_table["fold"]==i]), "симптомов без списка хороших", indication_drug_table[indication_drug_table["fold"]==i]["drugs_best"].value_counts()[0])

всего в фолде 145 симптомов без списка хороших 121
всего в фолде 145 симптомов без списка хороших 120
всего в фолде 145 симптомов без списка хороших 117
всего в фолде 144 симптомов без списка хороших 117
всего в фолде 144 симптомов без списка хороших 112


In [14]:
if USE_ONLY_MAIN_CONTEXT:
    indication_drug_table.to_csv("../data/interim/indication_popular_drug_mainContext.csv", index=False)
else:
    indication_drug_table.to_csv("../data/interim/indication_popular_drug.csv", index=False)
indication_drug_table

,indication,drugs,drugs_top5,drugs_best,fold
0,Повышенная температура тела,"парацетамол, виферон, анаферон, антигриппин, к...","парацетамол, виферон, анаферон, антигриппин, к...","лимон, гриппостад, фервекс для детей, колдакт ...",2
1,Угроза выкидыша,"аминокапроновая кислота, кислота аминокапронов...","аминокапроновая кислота, кислота аминокапронов...",нет хороших лекарств,5
2,Кровоизлияние,"метрогил, аминокапроновая кислота, кислота ами...","метрогил, аминокапроновая кислота, кислота ами...",нет хороших лекарств,4
3,Маточное кровотечение,"аминокапроновая кислота, кислота аминокапроновая","аминокапроновая кислота, кислота аминокапроновая",нет хороших лекарств,2
4,Стресс,"валериана, афобазол, глицин, пустырник, персен...","валериана, афобазол, глицин, пустырник, персен","релиум, микстура кватера, пустырника экстракт,...",2
...,...,...,...,...,...
718,"Гастроэнтерит, вызванный Escherichia coli",супракс,супракс,нет хороших лекарств,1
719,Синдром медиопателлярной складки,"нимесил, диклофенак, омез","нимесил, диклофенак, омез",нет хороших лекарств,5
720,Избыточная масса тела,кленбутерол,кленбутерол,нет хороших лекарств,5
721,заболевания ЖКТ,"корня солодкового, леденцы лакричные","корня солодкового, леденцы лакричные",нет хороших лекарств,1


# таблица комбинация симптомов; препарат;
coming soon... maybe

In [15]:
indication_drug_combo_dict = defaultdict(set)
indi_combo_counter = Counter()
drug_combo_counter = Counter()

for docData in corpus_ent_only:
    drugs = set()
    indications = set()
    for entity in docData:
        if entity.get("MedType", None)=="Drugname":
            drugs.add(Drug_normalization_dict[entity["text"].lower()]["standart"])
        if entity.get("DisType", None)=="Indication":
            if entity["MedDRA"].strip()!="":
                indications.add(entity["MedDRA"])
            else:
                indications.add(entity["text"])
    if len(indications)==0:
        continue
    indication_drug_combo_dict[tuple(sorted(list(indications)))].update(drugs)

# таблица препарат, атх, список заболеваний (мкб); список симптомов(meddra); спосок ADR(meddra)
coming soon... maybe